In [1]:
import pandas as pd
import pickle
import numpy as np
import tqdm
import pandas as pd
import json
import datetime

get_all_games = 1
start_val = 500

In [2]:
if get_all_games:
    query = """
    select * from `analytics-156605.barath.ludo_scores`
    """

    all_games = pd.read_gbq(query, use_bqstorage_api=True, progress_bar_type='tqdm_notebook')

    # all_games['time_stamp'] = pd.to_datetime(all_games['time_stamp'], format='%Y-%m-%d %H:%M:%S.%f')
    
    # all_games.sort_values(by='time_stamp')
    
    all_games = all_games.to_dict('records')

    # with open('../data/all_game_score_aug_16_to_sept_15.pkl', 'wb') as f:
    #     pickle.dump(all_games, f)

else:

    with open('../data/all_game_score_aug_16_to_sept_15.pkl', 'rb') as f:
        all_games = pickle.load(f)

Downloading:   0%|          | 0/918797 [00:00<?, ?rows/s]

In [3]:
for i in tqdm.tqdm(range(len(all_games))):
    all_games[i]['score_1'] = np.sum(json.loads(all_games[i]['score_1']))
    all_games[i]['score_2'] = np.sum(json.loads(all_games[i]['score_2']))

100%|██████████| 918797/918797 [00:16<00:00, 55902.61it/s]


In [4]:
live_scores = {}

for i in tqdm.tqdm(range(len(all_games))):
    data = all_games[i]

    if str(data['dt']) in ['2022-09-11', '2022-09-12', '2022-09-13', '2022-09-14', '2022-09-15']:

        if data['from_user'] not in live_scores:
            live_scores[data['from_user']] = start_val
        if data['opponent'] not in live_scores:
            live_scores[data['opponent']] = start_val

        all_games[i]['new_skill_1'] = live_scores[data['from_user']]
        all_games[i]['new_skill_2'] = live_scores[data['opponent']]

        live_scores[data['from_user']] += (data['score_1'] - data['score_2'])
        live_scores[data['opponent']] += (data['score_2'] - data['score_1'])

100%|██████████| 918797/918797 [00:03<00:00, 245929.84it/s]


In [5]:
win = []
q_pred = []
skill_pred = []
q_score_0 = []
q_score_1 = []

X = []
X_test = []
y = []
y_test = []

for i in tqdm.tqdm(range(len(all_games))):
    data = all_games[i]
    if str(data['dt']) in ['2022-09-11', '2022-09-12', '2022-09-13', '2022-09-14', '2022-09-15']:
        win.append(data['score_1'] > data['score_2'])
        q_pred.append(data['new_skill_1'] > data['new_skill_2'])
        skill_pred.append((data['mu'] - 3*data['sigma']) > (data['mu_opp'] - 3*data['sigma_opp']))

        q_score_0.append(data['new_skill_1'])
        q_score_1.append(data['new_skill_2'])

    if str(data['dt']) in ['2022-09-11', '2022-09-12', '2022-09-13', '2022-09-14']:
        X.append([data['mu'], data['sigma'], data['new_skill_1'], 
                        data['mu_opp'], data['sigma_opp'], data['new_skill_2']])
        y.append(data['score_1'] > data['score_2'])
    elif str(data['dt']) == '2022-09-15':
        X_test.append([data['mu'], data['sigma'], data['new_skill_1'], 
                        data['mu_opp'], data['sigma_opp'], data['new_skill_2']])
        y_test.append(data['score_1'] > data['score_2'])

100%|██████████| 918797/918797 [00:04<00:00, 207240.63it/s]


In [6]:
win = np.array(win).astype('int32')
q_pred = np.array(q_pred).astype('int32')
skill_pred = np.array(skill_pred).astype('int32')

q_score_0 = np.array(q_score_0)
q_score_1 = np.array(q_score_1)

bins = np.abs(q_score_0 - q_score_1)

X = np.array(X)
X_test = np.array(X_test)
y = np.array(y)
y_test = np.array(y_test)

In [7]:
u1 = []
tot = []
for i in range(25):
    idx = np.logical_and(bins>i*200, bins<(i+1)*200)
    temp = win[idx]
    u1.append(np.sum(temp[q_score_0[idx]>q_score_1[idx]]) + np.sum(temp[q_score_0[idx]<q_score_1[idx]]==0))
    tot.append(len(q_score_0[idx]))

In [8]:
import pandas as pd
pd.DataFrame(np.array([u1, tot]).T, columns=['1', '2'])

,1,2
0,23138,46397
1,18340,36613
2,13319,26256
3,9577,18886
4,7083,13774
5,5145,9985
6,3557,7074
7,2544,5022
8,1923,3794
9,1549,2894


In [9]:
# with open('../data/all_game_score_features_aug_16_to_sept_15.pkl', 'wb') as f:
#     pickle.dump([X, X_test, y, y_test], f)

In [10]:
from sklearn.preprocessing import StandardScaler
from sklearn.pipeline import Pipeline
from sklearn.linear_model import LogisticRegression
from sklearn.ensemble import GradientBoostingClassifier

In [11]:
clf = Pipeline([('scaler', StandardScaler()), ('gbc', GradientBoostingClassifier(random_state=42, verbose=1))])
# clf = Pipeline([('scaler', StandardScaler()), ('gbc', LogisticRegression(random_state=42, verbose=1))])
clf = clf.fit(X, y)

      Iter       Train Loss   Remaining Time 
         1           1.3771            3.27m
         2           1.3766            3.22m
         3           1.3762            3.20m
         4           1.3758            3.16m
         5           1.3755            3.12m
         6           1.3752            3.09m
         7           1.3750            3.05m
         8           1.3747            3.02m
         9           1.3745            2.98m
        10           1.3743            2.95m
        20           1.3730            2.61m
        30           1.3723            2.27m
        40           1.3719            1.94m
        50           1.3717            1.62m
        60           1.3715            1.29m
        70           1.3713           58.20s
        80           1.3711           38.84s
        90           1.3710           19.42s
       100           1.3709            0.00s


In [12]:
y_pred = clf.predict(X_test)
y_probs = clf.predict_proba(X_test)

In [13]:
from sklearn.metrics import accuracy_score, precision_score, recall_score, confusion_matrix, ConfusionMatrixDisplay

def perf(ytrue, ypred):
    accu = accuracy_score(ytrue, ypred)
    prec = precision_score(ytrue, ypred)
    rec = recall_score(ytrue, ypred)
    return accu*100, prec*100, rec*100

In [14]:
acc, precision, recall = perf(y_test, y_pred)
print(acc, precision, recall)

55.39586212854181 55.47575887032525 94.59560560718224


In [15]:
bins = np.abs(y_probs[:,1]-0.5)

In [16]:
u1 = []
tot = []
for i in range(25):
    idx = np.logical_and(bins>i*0.05, bins<(i+1)*0.05)
    temp = y_test[idx]
    u1.append(np.sum(temp[y_probs[:,1][idx]>=0.5]==1) + np.sum(temp[y_probs[:,1][idx]<0.5]==0))
    tot.append(len(y_test[idx]))

In [17]:
pd.DataFrame(np.array([u1, tot]).T, columns=['1', '2'])

,1,2
0,58375,108606
1,37464,65902
2,5185,8324
3,1534,2329
4,44,54
5,0,1
6,0,0
7,0,0
8,0,0
9,0,0
